# H A $\alpha$ decomposition evaluation

In this notebook we compare results of the new implementation of the
H A $\alpha$ decomposition with the C legacy version of PolSARpro.  
It assumes the user has a working installation of PolSARpro and has followed the 
instructions in the README.md file.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
# avoid thread conflicts between numpy and dask
# os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"


import numpy as np
from pathlib import Path
from polsarpro.io import read_PSP_bin, read_T3
from polsarpro.decompositions import h_a_alpha, h_a_alpha_dask
from pathlib import Path
import numpy as np
from spectral.io.envi import save_image

# change to your local C-PolSARpro install dir
c_psp_dir = "/home/c_psp/Soft/bin/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_process_sngl/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_convert/"

# change to your data paths
input_test_dir = Path("/data/psp/input/h_a_alpha_decomposition/")
# output_test_dir = Path("/data/res/freeman_cpsp")
output_test_dir = Path("/data/res/h_a_alpha_cpsp")
# output_test_dir = Path("/data/psp/output/freeman_decomposition/out")

## Run the C-version on some test data

In [ ]:
from polsarpro.devtools import parse_psp_parameter_string
import os
# fl1: alpha, beta, delta, lambda
# fl2: lambda
# fl3: alpha
# fl4: entropy 
# fl5: anisotropy
# fl6-9 combinations

input_str= f"""id: {input_test_dir} 
od: {output_test_dir}
iodf: T3
nwr: 7
nwc: 7
ofr: 0
ofc: 0
fnr: 18432
fnc: 1248
fl1: 1
fl2: 1
fl3: 1
fl4: 1
fl5: 1
fl6: 0
fl7: 0
fl8: 0
fl9: 0
errf: /tmp/MemoryAllocError.txt
mask: {input_test_dir}/mask_valid_pixels.bin
"""
result = parse_psp_parameter_string(input_str)
os.system(f"h_a_alpha_decomposition.exe {result}")

## Load coherency matrices and C-PSP outputs

In [ ]:
T3 = read_T3(input_test_dir)

out_names = ["entropy", "anisotropy", "alpha", "beta", "delta", "gamma", "lambda"]
out_c = {}

for name in out_names:
    file_name = output_test_dir / f"{name}.bin"
    out_c[name] = read_PSP_bin(file_name)

## Apply the numpy implementation (single-thread, only for reference)

In [ ]:
flags = ("entropy", "anisotropy", "alpha", "beta", "delta", "gamma", "lambda")
out_np = h_a_alpha(T3, "T3", boxcar_size=[7, 7], flags=flags)

## Apply the dask implementation (multi-thread)

In [ ]:
flags = ("entropy", "anisotropy", "alpha", "beta", "delta", "gamma", "lambda")
out_da = h_a_alpha_dask(T3, "T3", boxcar_size=[7, 7], flags=flags)

# write outputs
for name in out_da.keys():
    save_image(hdr_file=output_test_dir / f"{name}_py.hdr", image=out_da[name], force=True)

# Numerical evaluation

In [ ]:
for var in out_names:
    var_c = out_c[var]
    var_da = out_da[var]
    relerr = np.nanmean(((np.abs(var_da - var_c) / np.abs(var_c + var_da))))
    print(f"Variable {var}")
    print(f"Relative error between C and python: {relerr}")

In [ ]:
for var in out_names:
    var_np = out_np[var]
    var_da = out_da[var]
    err = np.nanmean(var_da - var_np)
    print(f"Variable {var}")
    print(f"Error between dask and numpy: {err}")